In [8]:
import tensorflow as tf
import numpy as np
import cv2
import PIL.Image as Image
import os
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import pathlib

In [9]:
Image_Shape = (224,224)

In [11]:
URL_dataset ="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

In [12]:
data_dir = tf.keras.utils.get_file(origin=URL_dataset,
fname='flower_photos',
untar=True)
data_dir = pathlib.Path(data_dir)

228813984/228813984 [==============================] - 38s 0us/step


In [13]:
# Total images
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [14]:
"""### Making the different classes"""
flowers_images_dict = {
    "daisy" : list(data_dir.glob('daisy/*')),
    "dandelion" : list(data_dir.glob('dandelion/*')),
    "roses" : list(data_dir.glob('roses/*')),
    "sunflowers" : list(data_dir.glob('sunflowers/*')),
    "tulips" : list(data_dir.glob('tulips/*'))
}


In [15]:
flowers_labels_dict= {
    "daisy" : 0,
    "dandelion" : 1,
    "roses" : 2,
    "sunflowers" : 3,
    "tulips" : 4
}

In [16]:
"""### Reshaping Images"""
X, Y = [],[]


In [17]:
for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, Image_Shape)
        X.append(resized_img)
        Y.append(flowers_labels_dict[flower_name])

In [18]:
X = np.array(X)
y = np.array(Y)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [20]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [21]:
"""### Pretrained Model"""
tf_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"


In [22]:
classifier = tf.keras.Sequential([
    hub.KerasLayer(tf_model,input_shape=(224,224,3), trainable=False),
    tf.keras.layers.Dense(len(flowers_labels_dict), activation="softmax")
])
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [23]:
classifier.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)


In [24]:
classifier.fit(X_train_scaled, y_train,epochs=5)

Epoch 1/5


C:\Users\vozon\anaconda3\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


86/86 [==============================] - 75s 802ms/step - loss: 0.8284 - accuracy: 0.6842
Epoch 2/5
86/86 [==============================] - 67s 774ms/step - loss: 0.4183 - accuracy: 0.8536
Epoch 3/5
86/86 [==============================] - 66s 768ms/step - loss: 0.3334 - accuracy: 0.8895
Epoch 4/5
86/86 [==============================] - 66s 764ms/step - loss: 0.2797 - accuracy: 0.9073
Epoch 5/5
86/86 [==============================] - 66s 768ms/step - loss: 0.2447 - accuracy: 0.9215


In [25]:
classifier.evaluate(X_test_scaled, y_test)

29/29 [==============================] - 25s 794ms/step - loss: 0.3564 - accuracy: 0.8715


[0.3563876152038574, 0.8714597225189209]

In [62]:
from PIL import Image
img = Image.open("daisy.jpg.crdownload")
img = tf.keras.preprocessing.image.img_to_array(img.resize(Image_Shape))
img = np.array([img])

In [63]:
res = classifier.predict(img)

1/1 [==============================] - 0s 70ms/step


In [64]:
print("The prediction is :{}".format(list(flowers_labels_dict.keys())[np.argmax(res)]))

The prediction is :daisy
